# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Answer Key

## Part C | Scrapping and Amazon Product Reviews (cont.)

> ## We are now ready to extract the reviews offline and no longer need to query the Amazon website.

In [1]:
import os
import gzip
import json
import lxml.html
import dateutil

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

> ## Input

In [2]:
with gzip.open(os.path.join('..', 'datasets', 'dataset-04-reviews.json.gz'), 'rb') as f:
    pages = json.loads(f.read())

In [3]:
len(pages)

2

## First page

In [4]:
page = pages['1']['content']

In [5]:
page

u'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script><!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'><link rel=\'dns-prefetch\' href=\'//images-na.ssl-images-amazon.com\'><link rel=\'dns-prefetch\' href=\'//m.media-amazon.com\'><link rel=\'dns-prefetch\' href=\'//completion.amazon.com\'><script type=\'text/javascript\'>\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){if(1==window.ueinit)try{return b.apply(t

In [6]:
document = lxml.html.fromstring(page)

In [7]:
type(document)

lxml.html.HtmlElement

(http://lxml.de/api/lxml.html-module.html#fromstring and http://lxml.de/api/lxml.html.HtmlElement-class.html)

> ## All reviews of a page

(http://lxml.de/api/lxml.etree._Element-class.html#xpath)

In [8]:
reviews = document.xpath('//*[@data-hook="review"]')

## First review

In [9]:
review = reviews[0]

In [10]:
type(review)

lxml.html.HtmlElement

> ## id

(http://lxml.de/api/lxml.etree._Element-class.html#get)

In [11]:
review.get('id')

'R29ISC3WOB5X8E'

> # star rating

In [12]:
review.find('.//*[@data-hook="review-star-rating"]').get('class')

'a-icon a-icon-star a-star-5 review-rating'

(http://lxml.de/api/lxml.etree._Element-class.html#find)

(https://en.wikipedia.org/wiki/XPath)

> ## title

In [13]:
review.findtext('.//*[@data-hook="review-title"]')

'Highly recommend'

(http://lxml.de/api/lxml.etree._Element-class.html#findtext)

> ## author

In [14]:
review.findtext('.//*[@data-hook="review-author"]/*[@data-hook="review-author"]')

'Mason selby'

> ## date

In [15]:
review.findtext('.//*[@data-hook="review-date"]')

'on July 4, 2017'

> ## body

In [16]:
review.findtext('.//*[@data-hook="review-body"]')

'I have bought silicon rings before and they are fragile. They always broke into two parts after only one or two weeks. I was suspicious of this one set. But after two weeks, it still holds on my finger. Really impressed. I will continute to observe their performance. I am satisfied so far and willing to recommend to my friends.'

> ## Output

In [17]:
df = pd.DataFrame(columns = ['date', 'id', 'author', 'title', 'body', 'star_rating'])

In [18]:
df

,date,id,author,title,body,star_rating


## Putting all of it together

(https://docs.python.org/2/howto/unicode.html and https://docs.python.org/2/library/stdtypes.html)

In [19]:
def date(node):
    date = review.findtext('.//*[@data-hook="review-date"]').replace('on ', '')
    return dateutil.parser.parse(date)

def id(node):
    return node.get('id')

def author(node):
    return node.findtext('.//*[@data-hook="review-author"]/*[@data-hook="review-author"]').encode('ascii', 'ignore')

def title(node):
    return node.findtext('.//*[@data-hook="review-title"]').encode('ascii', 'ignore')

def body(node):
    return node.findtext('.//*[@data-hook="review-body"]').encode('ascii', 'ignore')

def star_rating(node):
    node = node.find('.//*[@data-hook="review-star-rating"]')

    if node == None:
        return np.nan

    for star_rating in range(1, 6):
        if node.find_class('a-star-{:d}'.format(star_rating)):
             return star_rating

    return np.nan

(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html)

In [20]:
for i in sorted(pages.keys(), key = lambda i: int(i)):
    page = pages[i]['content']
    document = lxml.html.fromstring(page)
    reviews = document.xpath('//*[@data-hook="review"]')

    for review in reviews:
        df = df.append({'date': date(review),
                        'id': id(review),
                        'author': author(review),
                        'title': title(review),
                        'body': body(review),
                        'star_rating': star_rating(review)},
                       ignore_index = True)

In [21]:
df

,date,id,author,title,body,star_rating
0,2017-07-04,R29ISC3WOB5X8E,Mason selby,Highly recommend,I have bought silicon rings before and they ar...,5.0
1,2017-07-03,R9NVGZBOA9Q91,Jess Sweely,Feel and look great,I bought these to wear when I go to the gym to...,5.0
2,2017-07-03,R16G33K3JZ0CTC,Victor DeLaRosa,Worth the purchase,Excellent ring. I don't even notice it's on. I...,5.0
3,2017-07-03,R3KRKKNDHOV266,Akash deepgautam,Delicate and light,Lighter than I thought. Feel no burden. It loo...,5.0
4,2017-07-02,R2XB715AOXY8M6,VAN VO,Great rings!,I received my rings...They are wonderful. I lo...,5.0
...,...,...,...,...,...,...
12,2017-06-22,R3FKE63ZFILHDP,Matthew DeLara,I love these rings!,I love these rings! I always end up having to ...,5.0
13,2017-06-22,R3703RZSG3WMA7,H ome,perfect fit!,"Because of my work, I usually do not take my r...",5.0
14,2017-06-18,R3RRFHEULQAIA5,Amazon Customer,Just what I needed.,Excellent product. True to size. Comfortable.,5.0
15,2017-06-17,R2G7PVVEZ5AFJT,Amazon Customer,Five Stars,My husband loved it! Wears in place of his wed...,5.0


In [22]:
df.shape

(17, 6)

In [23]:
df.to_csv(os.path.join('..', 'datasets', 'dataset-04-reviews.csv'), index = False)